In [1]:
def get_input(fname="input.txt"):
    with open(fname) as f:
        return [int(n) for n in f.readline().strip().split(",")]

In [2]:
class Program(object):
    instructions = None
    input = None
    output = None
    is_halted = False
    idx = 0
    relative_base = 0
    
    STOPPED = -1
    NEEDS_INPUT = -2
    
    def __init__(self, instructions, input, output):
        self.instructions = {i: v for i, v in enumerate(instructions)}
        self.input = input
        self.output = output
    
    def _get_address(self, pos, mode=1):
        if mode == 0:
            return self.instructions.get(pos, 0)
        elif mode == 2:
            return self.instructions.get(pos, 0) + self.relative_base
    
    def _get_value(self, pos, mode=1):
        v = self.instructions.get(pos, 0)
        if mode == 0:
            v = self.instructions.get(v, 0)
        elif mode == 2:
            v = self.instructions.get(v + self.relative_base)
        return v
    
    def step(self):
        if self.is_halted:
            return Program.STOPPED
        opcode = self.instructions.get(self.idx, 0)
        op = opcode % 100
        opcode //= 100
        if op == 99: # halt
            self.is_halted = True
            return Program.STOPPED
        elif op == 9: # set relative base
            mode = opcode % 10
            v = self._get_value(self.idx + 1, mode)
            self.relative_base += v
            self.idx += 2
        elif op in (1, 2, 7, 8): # addition, multiplication, less than, equals
            modes = []
            for i in range(3):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b, idx_o = self.idx + 1, self.idx + 2, self.idx + 3
            mode_a, mode_b, mode_o = modes
            a = self._get_value(idx_a, mode_a)
            b = self._get_value(idx_b, mode_b)
            o = self._get_address(idx_o, mode_o)
            if op in (1, 2): # addition, multiplication
                res = a + b if op == 1 else a * b
            elif op == 7:
                res = int(a < b)
            elif op == 8:
                res = int(a == b)
            self.instructions[o] = res
            self.idx += 4
        elif op in (5, 6): # jump if true / false
            modes = []
            for i in range(2):
                modes.append(opcode % 10)
                opcode //= 10
            idx_a, idx_b = self.idx + 1, self.idx + 2
            mode_a, mode_b = modes
            a = self._get_value(idx_a, mode_a)
            b = self._get_value(idx_b, mode_b)
            if op == 5:
                if a != 0:
                    self.idx = b
                else:
                    self.idx += 3
            elif op == 6:
                if a == 0:
                    self.idx = b
                else:
                    self.idx += 3
        elif op == 3: # read input
            if len(self.input) == 0:
                return Program.NEEDS_INPUT
            mode = opcode % 10
            self.instructions[self._get_address(self.idx + 1, mode)] = self.input.popleft()
            self.idx += 2
        elif op == 4: # write output
            mode = opcode % 10
            output = self._get_value(self.idx + 1, mode)
            self.output.append(output)
            self.idx += 2
    

In [3]:
from collections import deque

In [4]:
input_q, output_q = deque(), deque()

In [5]:
p = Program(get_input(), input_q, output_q)

In [6]:
current_position = (0, 0)
UP, RIGHT, DOWN, LEFT = list(range(4))
current_direction = UP # 0 - UP, 1 - RIGHT, 2 - DOWN, 3 - LEFT

In [7]:
def move(position, direction):
    if direction == UP:
        return (position[0], position[1] - 1)
    if direction == RIGHT:
        return (position[0] + 1, position[1])
    if direction == DOWN:
        return (position[0], position[1] + 1)
    if direction == LEFT:
        return (position[0] - 1, position[1])

In [8]:
def get_color_at(colors, position, default=0):
    if position[0] not in colors:
        colors[position[0]] = {}
    if position[1] not in colors[position[0]]:
        colors[position[0]][position[1]] = default
    return colors[position[0]][position[1]]

In [9]:
def set_color_at(colors, position, color):
    if position[0] not in colors:
        colors[position[0]] = {}
    colors[position[0]][position[1]] = color

In [10]:
colors = { 0: { 0: 0 }}

In [11]:
visited = set()

In [12]:
while not p.is_halted:
    input_q.append(get_color_at(colors, current_position))
    while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
    if not p.is_halted:
        visited.add(current_position)
        direction_mv = output_q.pop()
        if direction_mv == 0:
            current_direction = (current_direction - 1) % 4
        else:
            current_direction = (current_direction + 1) % 4
        set_color_at(colors, current_position, output_q.pop())
        current_position = move(current_position, current_direction)

In [13]:
output_q

deque([0, 1])

In [14]:
len(visited)

2322

In [15]:
input_q, output_q = deque(), deque()

In [16]:
p = Program(get_input(), input_q, output_q)

In [17]:
colors = { 0: { 0: 1 }}

In [18]:
visited = set()

In [19]:
while not p.is_halted:
    input_q.append(get_color_at(colors, current_position, 1))
    while p.step() not in (Program.STOPPED, Program.NEEDS_INPUT): pass
    if not p.is_halted:
        visited.add(current_position)
        direction_mv = output_q.pop()
        if direction_mv == 0:
            current_direction = (current_direction - 1) % 4
        else:
            current_direction = (current_direction + 1) % 4
        set_color_at(colors, current_position, output_q.pop())
        current_position = move(current_position, current_direction)

In [20]:
min_x, max_x = min(colors.keys()), max(colors.keys())
min_y, max_y = min([k for c in colors.values() for k in c]), max([k for c in colors.values() for k in c])

In [21]:
min_x, max_x, max_x - min_x

(0, 60, 60)

In [22]:
min_y, max_y, max_y - min_y

(-41, 0, 41)

In [23]:
for i in reversed(range(min_y, 1 + max_y)):
    for j in reversed(range(min_x, 1 + max_x)):
        c = get_color_at(colors, (j, i), 1)
        if (j, i) not in visited:
            print(' ', end='')
        elif c == 1:
            print('█', end='')
        else:
            print(' ', end='')
    print()